# Train Legal Assertion



![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)





[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/16.Training_Legal_Assertion.ipynb)

# Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

👌 License detected in Environment Variables
🚨 Outdated Medical Secrets in license file. Version=4.0.0 but should be Version=0.1.14
🚨 Outdated OCR Secrets in license file. Version=4.0.0 but should be Version=4.1.0
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up if John Snow Labs home exists in /root/.johnsnowlabs this might take a few minutes.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



🚨 Cannot install 🕶Spark-OCR because provided license file secret is outdated
How to Fix this: 
Option1: Run jsl.install(force_browser==True) to get a Browser Window pop-up where you can refresh your license data.
Option2: Run jsl.install(secrets_file=="path/to/fresh_credentials.json") after downloading a fresh license from https://my.johnsnowlabs.com/subscriptions   
Option3: Run jsl.install(refresh_install=True,force_browser==True) to refresh everything. 
Traceback (most recent call last):
ValueError: Trying to download Invalid URL! https://pypi.johnsnowlabs.com/4.0.0-49cdb09f66ca01a93f959366f0e4a84d1a09b2df/spark-ocr/spark_ocr-4.1.0-py3-none-any.whl

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
Exception: 🚨 Cannot install 🕶Spark-OCR because provided license file secret is outdated
How to Fix this: 
Option1: Run jsl.install(force_browser==True) to get a Browser Window pop-up where you can refresh your license data.
Option2: Ru

In [ ]:
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

DEBUG START!
📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
🤓 Looks like /root/.johnsnowlabs is missing, creating it
👷 Setting up if John Snow Labs home exists in /root/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
🤓 Looks like you are missing some jars, trying fetching them ...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



🚨 Cannot install 🕶Spark-OCR because provided license file secret is outdated
How to Fix this: 
Option1: Run jsl.install(force_browser==True) to get a Browser Window pop-up where you can refresh your license data.
Option2: Run jsl.install(secrets_file=="path/to/fresh_credentials.json") after downloading a fresh license from https://my.johnsnowlabs.com/subscriptions   
Option3: Run jsl.install(refresh_install=True,force_browser==True) to refresh everything. 
Traceback (most recent call last):
ValueError: Trying to download Invalid URL! https://pypi.johnsnowlabs.com/4.0.0-49cdb09f66ca01a93f959366f0e4a84d1a09b2df/jars/spark-ocr-assembly-4.1.0.jar

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
Exception: 🚨 Cannot install 🕶Spark-OCR because provided license file secret is outdated
How to Fix this: 
Option1: Run jsl.install(force_browser==True) to get a Browser Window pop-up where you can refresh your license data.
Option2: Run jsl.ins

In [ ]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml import Pipeline
from sparknlp import Doc2Chunk
from sparknlp_jsl.annotator import *
import sparknlp_jsl
from sparknlp.base import LightPipeline

In [ ]:
spark

# Data Prep 

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Legal/data/aserrtion_fin.csv

In [ ]:
import pandas as pd

training_df = pd.read_csv('/content/assertion_fin.csv')

In [ ]:
training_data = spark.createDataFrame(training_df)
training_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
|task_id|            sentence|tkn_start|tkn_end|               chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
|      1|The Swedish East ...|        1|      4|Swedish East Indi...|   ORG|           PAST|
|      1|The Swedish East ...|        6|      8|Svenska Ostindisk...| ALIAS|           PAST|
|      1|The Swedish East ...|       10|     10|                SOIC| ALIAS|           PAST|
|      1|The Swedish East ...|       14|     14|          Gothenburg|   LOC|           PAST|
|      1|The Swedish East ...|       15|     15|              Sweden|   LOC|           PAST|
|      1|The Swedish East ...|       17|     17|                1731|  DATE|           PAST|
|      1|The Swedish East ...|       25|     25|               China|   LOC|           PAST|
|      1|The Swedish East ...|       28|     29|            Far East| 

In [ ]:
training_data.printSchema()

root
 |-- task_id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- tkn_start: long (nullable = true)
 |-- tkn_end: long (nullable = true)
 |-- chunk: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- assertion_label: string (nullable = true)



In [ ]:
%time training_data.count()

CPU times: user 11.9 ms, sys: 0 ns, total: 11.9 ms
Wall time: 878 ms


8050

In [ ]:
(train_data, test_data) = training_data.randomSplit([0.9, 0.1], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 8050
Test Dataset Count: 802


In [ ]:
train_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
|task_id|            sentence|tkn_start|tkn_end|               chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
|      1|"Stockholms-varve...|        6|      6|           Stockholm|   LOC|           PAST|
|      1|"The funny busine...|        5|      8|Swedish East Indi...|   ORG|           PAST|
|      1|             (1998).|        0|      0|                1998|  DATE|           PAST|
|      1|2.5 tonnes) and t...|       34|     34|              Sweden|   LOC|           PAST|
|      1|37. Gothenburg: R...|        2|      7|Royal Society of ...|   ORG|           PAST|
|      1|= Decline and fal...|       11|     11|                1806|  DATE|           PAST|
|      1|= Early attempts ...|        9|     11|  Swedish East India|   ORG|           PAST|
|      1|= Early attempts ...|       19|     19|            merchant| 

# Using RoBerta Embeddings

In [ ]:
roberta_embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]


In [ ]:
document = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("doc_chunk")\
    .setChunkCol("chunk")\
    .setStartCol("tkn_start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(False)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')


We save the test data in parquet format to use in `AssertionDLApproach()`. 

In [ ]:
assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    roberta_embeddings])

assertion_test_data = assertion_pipeline.fit(test_data).transform(test_data)

In [ ]:
assertion_test_data.columns

['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

In [ ]:
assertion_test_data.write.mode('overwrite').parquet('test_data.parquet')

In [ ]:
assertion_train_data = assertion_pipeline.fit(training_data).transform(training_data)
assertion_train_data.write.mode('overwrite').parquet('train_data.parquet')

In [ ]:
assertion_train_data.columns

['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

## Graph setup

In [ ]:
!pip install -q tensorflow==2.7.0
!pip install -q tensorflow-addons

     |████████████████████████████████| 665.5 MB 23 kB/s 
     |████████████████████████████████| 1.3 MB 14.2 MB/s 
     |████████████████████████████████| 463 kB 77.1 MB/s 
     |████████████████████████████████| 1.1 MB 14.9 MB/s 


We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline. 

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [ ]:
graph_folder= "./tf_graphs"

In [ ]:
assertion_graph_builder =  TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("assertion_label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(1200)\
    .setHiddenUnitsNumber(25)

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [ ]:
scope_window = [50, 50]

assertionStatus = legal.AssertionDLApproach()\
    .setLabelCol("assertion_label")\
    .setInputCols("document", "doc_chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setEpochs(2)\
    .setStartCol("tkn_start")\
    .setEndCol("tkn_end")\
    .setMaxSentLen(1200)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)
    #.setValidationSplit(0.2)\    
    #.setDropout(0.1)\    

In [ ]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    #document,
    #chunk,
    #token,
    #embeddings,
    assertion_graph_builder,
    assertionStatus])

In [ ]:
training_data.printSchema()

root
 |-- task_id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- tkn_start: long (nullable = true)
 |-- tkn_end: long (nullable = true)
 |-- chunk: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- assertion_label: string (nullable = true)



In [ ]:
assertion_train_data = spark.read.parquet('train_data.parquet')

In [ ]:
%%time
assertion_model = clinical_assertion_pipeline.fit(assertion_train_data)

Medical Graph Builder configuration:
Model name: assertion_dl
Graph folder: ./tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 4, 'feat_size': 768, 'max_seq_len': 1200, 'n_hidden': 25}


Instructions for updating:
non-resource variables are not supported in the long term


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

assertion_dl graph exported to ./tf_graphs/assertion_graph.pb
CPU times: user 13.5 s, sys: 1.57 s, total: 15 s
Wall time: 18min 26s


Checking the results saved in the log file

In [ ]:
import os

log_files = os.listdir("/content/training_logs")
log_files

['AssertionDLApproach_e8cd7fb0378e.log']

In [ ]:
with open("/content/training_logs/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ./tf_graphs/assertion_graph.pb
Training started, trainExamples: 8050


Epoch: 0 started, learning rate: 0.001, dataset size: 8050
Done, 494.821643681 total training loss: 49.79505, avg training loss: 0.79039764, batches: 63
Quality on test dataset: 
time to finish evaluation: 41.38s
Total test loss: 1.7628	Avg test loss: 0.2518
label	 tp	 fp	 fn	 prec	 rec	 f1
PRESENT	 226	 39	 15	 0.8528302	 0.93775934	 0.89328057
POSSIBLE	 173	 18	 15	 0.90575916	 0.92021275	 0.9129288
FUTURE	 105	 5	 15	 0.95454544	 0.875	 0.91304344
PAST	 218	 18	 35	 0.9237288	 0.86166006	 0.8916155
tp: 722 fp: 80 fn: 80 labels: 4
Macro-average	 prec: 0.9092159, rec: 0.89865804, f1: 0.9039061
Micro-average	 prec: 0.90024936, rec: 0.90024936, f1: 0.9002493


Epoch: 1 started, learning rate: 9.5E-4, dataset size: 8050
Done, 486.142873849 total training loss: 15.336851, avg training loss: 0.24344209, batches: 63
Quality on test dataset: 
time to finish evaluation: 42.45s
Total test loss: 1

In [ ]:
assertion_test_data = spark.read.parquet('test_data.parquet')

In [ ]:
preds = assertion_model.transform(assertion_test_data).select('assertion_label','assertion.result')

preds.show()

+---------------+---------+
|assertion_label|   result|
+---------------+---------+
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|        PRESENT|[PRESENT]|
|           PAST|   [PAST]|
|        PRESENT|[PRESENT]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
|        PRESENT|[PRESENT]|
|        PRESENT|[PRESENT]|
|        PRESENT|   [PAST]|
|           PAST|   [PAST]|
|           PAST|   [PAST]|
+---------------+---------+
only showing top 20 rows



In [ ]:
preds_df = preds.toPandas()

In [ ]:
preds_df["result"] = preds_df["result"].apply(lambda x: x[0] if len(x) else pd.NA)
preds_df.dropna(inplace=True)

preds_df

,assertion_label,result
0,PAST,PAST
1,PAST,PAST
2,PAST,PAST
3,PRESENT,PRESENT
4,PAST,PAST
...,...,...
797,PRESENT,PRESENT
798,PRESENT,PRESENT
799,PRESENT,PRESENT
800,PRESENT,PRESENT


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['assertion_label'], preds_df['result']))

              precision    recall  f1-score   support

      FUTURE       0.97      0.95      0.96       120
        PAST       0.92      0.98      0.95       250
    POSSIBLE       0.97      0.94      0.96       188
     PRESENT       0.96      0.93      0.95       241

    accuracy                           0.95       799
   macro avg       0.96      0.95      0.95       799
weighted avg       0.95      0.95      0.95       799



### Saving the trained model

In [ ]:
assertion_model.stages

In [ ]:
# Save a Spark NLP model
assertion_model.stages[-1].write().overwrite().save('Assertion')

# cd into saved dir and zip
! cd /content/Assertion ; zip -r /content/Assertion.zip *

  adding: fields/ (stored 0%)
  adding: fields/datasetParams/ (stored 0%)
  adding: fields/datasetParams/.part-00003.crc (deflated 45%)
  adding: fields/datasetParams/.part-00002.crc (stored 0%)
  adding: fields/datasetParams/part-00000 (deflated 26%)
  adding: fields/datasetParams/.part-00000.crc (stored 0%)
  adding: fields/datasetParams/part-00002 (deflated 27%)
  adding: fields/datasetParams/._SUCCESS.crc (stored 0%)
  adding: fields/datasetParams/part-00003 (deflated 95%)
  adding: fields/datasetParams/part-00001 (deflated 27%)
  adding: fields/datasetParams/.part-00001.crc (stored 0%)
  adding: fields/datasetParams/_SUCCESS (stored 0%)
  adding: metadata/ (stored 0%)
  adding: metadata/part-00000 (deflated 40%)
  adding: metadata/.part-00000.crc (stored 0%)
  adding: metadata/._SUCCESS.crc (stored 0%)
  adding: metadata/_SUCCESS (stored 0%)
  adding: tensorflow (deflated 40%)
